# Nulls in DataFrames


## What is `NULL`?

Before we start comparing various DataFrame Frameworks and Databases, we should first define what `NULL` is.

The answer actually depends on who you ask.

If you asked:
 * A Mathematician: They would probably point you to the [Null Set](https://en.wikipedia.org/wiki/Null_set)
 * A Programmer:  A null pointer or null reference is a value saved for indicating that the pointer or reference does not refer to a valid object. Which is the case for Static Typed langauges like C/C++ as well as Dynamic ones like Python where it is called `None`. [Wikipedia](https://en.wikipedia.org/wiki/Null_pointer)
 * A Database Geek: `NULL` is a special marker used to indicate that a data value does not exist. [Wikipedia](https://en.wikipedia.org/wiki/Null_(SQL))
 
 
The latter two definitions are the ones relevent to us. 

At first glance the Programmer's definition and the Database Geek's seem like the same thing and for many years I thought they were. However there's actually some deep implications using them interchangeably especially now that we use tools like Python DataFrame libaries such as Pandas, Spark and Daft, where databases would historically be used.


I think Kenneth Baclawski summarized the difference beautifully in his [write up](https://arxiv.org/html/1606.00740v1) on this topic:

*The main distinction between the relational null and the programming language null is the following: The relational null represents the absence of a value in a field of a record; whereas the programming language null represents one of the possible values of a variable. Succinctly,*

***The programming language null is a value but database null is not a value.*** 

*Consequently, the phrase "null value" is an oxymoron for databases.*

## Who is affect by this?

It matters if you want to deal with missing data!

Dealing with missing data is fairly common and can happen for a variety of reasons including that data actually being missing, doing left/right joins or doing a certain types of groupbys.

The problem however is that DataFrames sit in this awkward spot between Databases and Programming. So which Null should they use and what should the behavior be?


## Where does it matter?

In the following example, we go into a case where we have a dataset of movies and their review score. 

A new movie that just came out would be unreviewed and would be considered as missing data.

Let's consider the following schema for this dataset:

| title (string) | year (int) |  score (float)|
|---|---|---|


### SQLite Example

Let's load a mini-dataset into SQLite of all valid data and see what it looks like!

#### Without Nulls

In [1]:
import sqlite3
import tempfile 
from pprint import pprint

tfile = tempfile.mkstemp()[1]
con = sqlite3.connect(f"{tfile}.db")
cur = con.cursor()

# Create table with the schema defined above
cur.execute("CREATE TABLE movie(title, year, score)")

# Historical Movies
data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brian", 1979, 8.0),

]

cur.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
con.commit()

pprint(cur.execute(
    "SELECT * FROM movie where score != 7.5;"
).fetchall())

[('Monty Python Live at the Hollywood Bowl', 1982, 7.9),
 ("Monty Python's Life of Brian", 1979, 8.0)]


#### Now let's throw some Nulls in!

In [2]:
new_movie_without_a_rating = (
    "Monty Python's NULL adventure", 2022, None
)
cur.execute("INSERT INTO movie VALUES(?, ?, ?)", 
            new_movie_without_a_rating)
con.commit()

pprint(cur.execute(
    "SELECT * FROM movie where score != 7.5;"
).fetchall())

[('Monty Python Live at the Hollywood Bowl', 1982, 7.9),
 ("Monty Python's Life of Brian", 1979, 8.0)]


### Pandas Example

#### Let's load our dataset into Pandas now

In [3]:
import pandas as pd
df = pd.read_sql_query('SELECT * from movie;', con)

df

,title,year,score
0,Monty Python Live at the Hollywood Bowl,1982,7.9
1,Monty Python's The Meaning of Life,1983,7.5
2,Monty Python's Life of Brian,1979,8.0
3,Monty Python's NULL adventure,2022,NaN


#### And run the same `where` as before

In [4]:
df[df['score'] != 7.5]


,title,year,score
0,Monty Python Live at the Hollywood Bowl,1982,7.9
2,Monty Python's Life of Brian,1979,8.0
3,Monty Python's NULL adventure,2022,NaN


#### We get a different result!! Why?

Pandas default behavior treats Nulls/Nones as a value rather than a marker suggested by the SQL standard.

Pandas here is using the programming Null rather than the database Null!

In [5]:
df['score'].iloc[-1] == 7.5

False

#### How do I fix this?

Pandas actually can support proper Nulls but it is not enabled by default!

In [15]:
score_with_null = pd.Series([1, 7.5, 8.0, None], dtype='Float64')
score_with_null

0     1.0
1     7.5
2     8.0
3    <NA>
dtype: Float64

In [17]:
score_with_null.iloc[-1] == 7.5

<NA>

In [22]:
score_with_null.iloc[-1] == float('nan')

<NA>

This supports a null result rather than a boolean value!

Now let's swap out the `float` Series with a `Float` Series

In [18]:
df['score'] = score_with_null

In [19]:
df[df['score'] != 7.5]

,title,year,score
0,Monty Python Live at the Hollywood Bowl,1982,1.0
2,Monty Python's Life of Brian,1979,8.0


But how could I start with the right way!

In [20]:
df = pd.read_sql_query('SELECT * from movie;', con, dtype={'score': 'Float64'})

df

,title,year,score
0,Monty Python Live at the Hollywood Bowl,1982,7.9
1,Monty Python's The Meaning of Life,1983,7.5
2,Monty Python's Life of Brian,1979,8.0
3,Monty Python's NULL adventure,2022,<NA>


### Thank you and please checkout https://getdaft.io for an Awesome Distributed DataFrame Library!

#### Github Stars are appreciated! 